In [30]:
# http://gouthamanbalaraman.com/blog/american-option-pricing-quantlib-python.html
# https://quantlib-python-docs.readthedocs.io/en/latest/instruments/options.html
import QuantLib as ql
import matplotlib.pyplot as plt
import pandas as pd
import datetime
%matplotlib inline
ql.__version__

'1.23'

In [ ]:
# iv http://options-scanner.com/index.php?page=atm-implied-volatility
# https://warrants.ubs.com/en/warrants/warrants_analyzer_performance/code/11051
iv = 

In [49]:
df = pd.read_csv('./data/0914.options.csv')
df.head()

,type,underlying,strike,dividendYield,riskFreeRate,expiry,volatility
0,call,39.55,39,6.48,0.001,29/6/22,17.03


In [60]:
anhch = df.to_dict(orient='records')[0]
anhch

{'type': 'call',
 'underlying': 39.55,
 'strike': 39,
 'dividendYield': 6.48,
 'riskFreeRate': 0.001,
 'expiry': '29/6/22',
 'volatility': 17.03}

In [41]:
def get_o_type(type):
    if (type == 'call'):
        return ql.Option.Call
    elif (type == 'put'):
        return ql.Option.Put

In [68]:
def calc_option_price(otype, spot, strike, dyield, rfr, exp, iv):
    d,m,y = exp.split('/')
    maturity_date = ql.Date(int(d),int(m),int(y) + 2000)
    spot_price = spot
    strike_price = strike
    volatility = iv / 100 # the historical vols or implied vols
    dividend_rate = dyield / 100
    option_type = get_o_type(otype)
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    print(payoff)

In [69]:
calc_option_price(anhch['type'],
                  anhch['underlying'],
                  anhch['strike'],
                  anhch['dividendYield'],
                  anhch['riskFreeRate'],
                  anhch['expiry'],
                  anhch['volatility']
                 )

<QuantLib.QuantLib.PlainVanillaPayoff; proxy of <Swig Object of type 'ext::shared_ptr< PlainVanillaPayoff > *' at 0x7fb5d1da6060> >
